# The Hong Kong University of Science and Technology
# MSBD5002: Data Mining and Knowledge Discovery
# Fall 2021 Assignment 2

### Student name: Mak Chun Wai, Michael
### HKUST account: cwmakah
### Student ID: 20801333

#### 2. Implementation of Adaboost

#### Adaboost

In [1]:
# import necessary libraries
import pandas as pd
import random
import numpy as np

In [2]:
# Load training dataset and have a quick review
def clean_df():
    df = pd.DataFrame(columns=['#', 'x', 'y'])
    df['#'] = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    df['x'] = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
    df['y'] = [1, 1, -1, -1, -1, 1, 1, -1, -1, 1]
    df = df.set_index(['#'])
    df['w1'] = 1/(df.shape[0]) # Initialize weight as 1/N
    return df

In [3]:
# Have a quick look on the structure of data given
df = clean_df()
df

,x,y,w1
#,,,
1,0,1,0.1
2,1,1,0.1
3,2,-1,0.1
4,3,-1,0.1
5,4,-1,0.1
6,5,1,0.1
7,6,1,0.1
8,7,-1,0.1
9,8,-1,0.1


In [4]:
# Set up for iteration
thres = list(np.arange(0, 9.5, 0.5)) # Limit threshold be multiple of 0.5
df_original = clean_df()
polarity = [1, -1]
iteration = 10 # Can adjust the number of iterations by changing variable here
# Iterate 
for i in range(1, iteration+1):
    best_t = None
    best_polarity = None
    lowest_er = 1
    # Obtain 10 random samples depending on weights
    random.seed(10)
    df_sample = df_original.sample(len(df_original), replace = True, weights = df['w{}'.format(i)])
    # print(df_sample.head(10))
    # Obtain the best classifier between 0-9
    for t in thres:
        for p in polarity:
            df_sample['y_pred'] = np.sign((df_sample['x']-t)*p)
            df_sample['y_pred'] = df_sample['y_pred'].replace(0, p) # self-definition: if x = threshold; y = 1
            df_sample['misclass'] = abs(df_sample['y_pred']-df_sample['y'])/2
            # print(df_sample)
            er = sum(df_sample['misclass']*df_sample['w{}'.format(i)])/df_sample['x'].count()
            # print(er)
            if er < lowest_er:
                lowest_er = er
                best_t = t
                best_polarity = p
    # Print threshold and polarity for reporting
    print('t{} ='.format(i), best_t)
    print('p{} ='.format(i), best_polarity)
    # Learn by the best classifier
    df['y_pred{}'.format(i)] = np.sign((df['x']-best_t)*best_polarity)
    df['y_pred{}'.format(i)] = df['y_pred{}'.format(i)].replace(0, 1)
    df['misclass{}'.format(i)] = abs(df['y_pred{}'.format(i)]-df['y'])/2
    # Calculate error rate of the best classifier in the round
    er = sum(df['misclass{}'.format(i)]*df['w{}'.format(i)])/df['x'].count()
    # Calculate importance of the classifier in the round
    df['alpha{}'.format(i)] = np.log((1-er)/er)/2
    # Update weight
    new_weight = df['w{}'.format(i)]*np.exp(-1*df['alpha{}'.format(i)]*df['y']*df['y_pred{}'.format(i)])
    # Normalization
    df['w{}'.format(i+1)] = new_weight/sum(new_weight)
    df_original['w{}'.format(i+1)] = df['w{}'.format(i+1)]
df_original.head(10)

t1 = 6.5
p1 = -1
t2 = 4.5
p2 = 1
t3 = 1.5
p3 = -1
t4 = 0.0
p4 = 1
t5 = 8.5
p5 = 1
t6 = 6.5
p6 = -1
t7 = 4.5
p7 = 1
t8 = 1.5
p8 = -1
t9 = 0.0
p9 = 1
t10 = 8.5
p10 = 1


,x,y,w1,w2,w3,w4,w5,w6,w7,w8,w9,w10,w11
#,,,,,,,,,,,,,
1,0,1,0.1,0.009804,0.228007,0.020816,0.001909,0.169564,0.015464,0.375367,0.034365,0.003135,0.172306
2,1,1,0.1,0.009804,0.228007,0.020816,0.001909,0.169564,0.015464,0.375367,0.034365,0.003135,0.172306
3,2,-1,0.1,0.235294,0.021544,0.001967,0.037765,0.003440,0.180576,0.016529,0.001513,0.071410,0.006511
4,3,-1,0.1,0.235294,0.021544,0.001967,0.037765,0.003440,0.180576,0.016529,0.001513,0.071410,0.006511
5,4,-1,0.1,0.235294,0.021544,0.001967,0.037765,0.003440,0.180576,0.016529,0.001513,0.071410,0.006511
6,5,1,0.1,0.009804,0.000898,0.035032,0.003212,0.285363,0.026024,0.002382,0.056413,0.005147,0.282857
7,6,1,0.1,0.009804,0.000898,0.035032,0.003212,0.285363,0.026024,0.002382,0.056413,0.005147,0.282857
8,7,-1,0.1,0.009804,0.228007,0.020816,0.399681,0.036402,0.003320,0.080585,0.007378,0.348151,0.031746
9,8,-1,0.1,0.009804,0.228007,0.020816,0.399681,0.036402,0.003320,0.080585,0.007378,0.348151,0.031746


In [5]:
# Prediction based on the above result
t =[]
result = pd.DataFrame()
final_result = pd.DataFrame()
for i in range(1, iteration+1):
    result[i] = df['alpha{}'.format(i)]*df['y_pred{}'.format(i)]
    final_result['sum'] = result.sum(axis=1)
final_result['sign'] = np.sign(list(final_result['sum']))
final_result['label'] = list(clean_df()['y'])
final_result

,sum,sign,label
#,,,
1,4.168869,1.0,1
2,4.168869,1.0,1
3,-7.444586,-1.0,-1
4,-7.444586,-1.0,-1
5,-7.444586,-1.0,-1
6,3.673201,1.0,1
7,3.673201,1.0,1
8,-5.860386,-1.0,-1
9,-5.860386,-1.0,-1
